In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/Users/franco.benites/dm2025b/src/workflows/experiments_tracking.csv")

In [ ]:
df.rename(columns={"Metodo Drift": "drift"}, inplace=True)

In [ ]:
#change values to lower_case y snake_case
df.drift = df.drift.str.lower()
df.drift = df.drift.str.replace(" ", "_")

In [ ]:
df.head()

,Experimento,date,publicScore,envios,semilla,num_leaves,min_data_in_leaf,feature_fraction,learning_rate,num_iterations,drift
0,9105,NaN,7.506,1900.0,100379,256.0,4940.0,1,0.1,1139.0,ipc
1,9107,NaN,7.316,1900.0,101111,241.0,2727.0,1,0.1,779.0,ipc
2,9108,NaN,6.715,1800.0,250007,216.0,4912.0,1,0.1,207.0,ipc
3,9109,NaN,8.136,2000.0,700001,255.0,5247.0,1,0.1,859.0,ipc
4,9110,NaN,7.836,2400.0,800011,216.0,7393.0,1,0.1,454.0,ipc


In [ ]:
drift_methods = df.drift.unique().tolist()
print(drift_methods)

['ipc', 'nucleo', 'dolar_blue', 'ninguno', 'rank_simple']


In [ ]:
ipc = df[df.drift == 'ipc']['publicScore']
nucleo = df[df.drift == 'nucleo']['publicScore']
dolar_blue = df[df.drift == 'dolar_blue']['publicScore']
ninguno = df[df.drift == 'ninguno']['publicScore']
rank_simple = df[df.drift == 'rank_simple']['publicScore']

# R

In [16]:
library(data.table)

# Read the CSV
df <- fread("/Users/franco.benites/dm2025b/src/workflows/experiments_tracking.csv")

# Rename column
setnames(df, "Metodo Drift", "drift")

# Convert to lower case and replace spaces with underscores
df[, drift := tolower(drift)]
df[, drift := gsub(" ", "_", drift)]

df <- df[!is.na(df$publicScore)]

# Print head of data
head(df)

# Get unique drift methods
drift_methods <- unique(df$drift)
print(drift_methods)

# Create separate vectors for each drift method
score_by_drift <- list(
  ipc = df[drift == "ipc", publicScore],
  nucleo = df[drift == "nucleo", publicScore],
  dolar_blue = df[drift == "dolar_blue", publicScore],
  ninguno = df[drift == "ninguno", publicScore],
  rank_simple = df[drift == "rank_simple", publicScore]
)


Experimento,date,publicScore,envios,semilla,num_leaves,min_data_in_leaf,feature_fraction,learning_rate,num_iterations,drift
<int>,<dttm>,<dbl>,<int>,<int>,<int>,<int>,<int>,<dbl>,<int>,<chr>
9105,NA,7.506,1900,100379,256,4940,1,0.1,1139,ipc
9107,NA,7.316,1900,101111,241,2727,1,0.1,779,ipc
9108,NA,6.715,1800,250007,216,4912,1,0.1,207,ipc
9109,NA,8.136,2000,700001,255,5247,1,0.1,859,ipc
9110,NA,7.836,2400,800011,216,7393,1,0.1,454,ipc
9111,NA,6.225,1800,100379,256,2967,1,0.1,746,nucleo


[1] "ipc"         "nucleo"      "dolar_blue"  "ninguno"     "rank_simple"


In [ ]:
wt <- wilcox.test(score_by_drift$ipc, score_by_drift$nucleo)
pvalue <- wt$p.value
out <- 0
if (pvalue < 0.05 && mean(score_by_drift$ipc) > mean(score_by_drift$nucleo)) {
  out <- 1
} else if (pvalue < 0.05 && mean(score_by_drift$ipc) < mean(score_by_drift$nucleo)) {
  out <- 2
}
print(out)

In [18]:
# Get all unique pairs of methods
methods <- names(score_by_drift)
pairs <- expand.grid(method1 = methods, method2 = methods, stringsAsFactors = FALSE)  # Prevent factor conversion
pairs <- pairs[pairs$method1 < pairs$method2, ]  # Keep only unique pairs

# Function to compare two methods
compare_methods <- function(method1, method2) {
  # Convert to numeric if needed
  scores1 <- as.numeric(score_by_drift[[method1]])
  scores2 <- as.numeric(score_by_drift[[method2]])
  
  wt <- wilcox.test(scores1, scores2)
  pvalue <- wt$p.value
  
  out <- "NOT-SIGNIFICANT"
  if (pvalue < 0.05 && mean(scores1) > mean(scores2)) {
    out <- "1 WINS"  # method1 significantly better
  } else if (pvalue < 0.05 && mean(scores1) < mean(scores2)) {
    out <- "2 WINS"  # method2 significantly better
  }
  
  return(data.frame(
    method1 = method1,
    method2 = method2,
    pvalue = pvalue,
    mean1 = mean(scores1),
    mean2 = mean(scores2),
    result = out
  ))
}

# Apply comparison to all pairs
results <- do.call(rbind, 
  lapply(1:nrow(pairs), function(i) {
    compare_methods(pairs$method1[i], pairs$method2[i])
  })
)

# Print results in a nice format
print(results)

Warning message in wilcox.test.default(scores1, scores2):
"cannot compute exact p-value with ties"
Warning message in wilcox.test.default(scores1, scores2):
"cannot compute exact p-value with ties"


      method1     method2     pvalue  mean1   mean2          result
1  dolar_blue         ipc 0.21874201 6.9455 7.50180 NOT-SIGNIFICANT
2         ipc      nucleo 0.05555556 7.5018 6.79300 NOT-SIGNIFICANT
3  dolar_blue      nucleo 0.41269841 6.9455 6.79300 NOT-SIGNIFICANT
4     ninguno      nucleo 0.69047619 7.0094 6.79300 NOT-SIGNIFICANT
5         ipc     ninguno 0.22222222 7.5018 7.00940 NOT-SIGNIFICANT
6  dolar_blue     ninguno 1.00000000 6.9455 7.00940 NOT-SIGNIFICANT
7         ipc rank_simple 0.01587302 7.5018 5.50175          1 WINS
8      nucleo rank_simple 0.08509993 6.7930 5.50175 NOT-SIGNIFICANT
9  dolar_blue rank_simple 0.11428571 6.9455 5.50175 NOT-SIGNIFICANT
10    ninguno rank_simple 0.03174603 7.0094 5.50175          1 WINS
